In [10]:
import cv2
import imutils
from imutils.video import VideoStream
import dlib
import scipy
import argparse
import numpy as np
import PIL.Imag
'''
All packages imported successfully.
'''

'\nAll packages imported successfully.\n'

## LAB color space
#### L* for the lightness from black (0) to white (100), a* from green (−) to red (+), and b* from blue (−) to yellow (+).

In [ ]:
# print("[INFO] starting video stream thread...")
# vs = VideoStream(src=arg["webcam"]).start()
# time.sleep(1.0)
 
# # loop over frames from the video stream
# while True:
# 	# grab the frame from the threaded video file stream, resize
# 	# it, and convert it to grayscale
# 	# channels)
# 	frame = vs.read()
# 	frame = imutils.resize(frame, width=450)
# 	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
 
# 	# detect faces in the grayscale frame
# 	rects = detector(gray, 0)
    
# cv2.imshow("output",frame)
# cv2.waitKey(0)
# cv2.destroyWindow('img')

In [25]:
cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Display the resulting frame
    cv2.imshow('frame',gray)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyWindow('frame')

#### Processing image filtering and contrast on video frames

In [60]:
cap = cv2.VideoCapture(0)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
#     lab = cv2.cvtColor(frame,cv2.COLOR_BGR2LAB)
#     l,a,b = cv2.split(lab)
#     clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(12,12))
#     cl = clahe.apply(l)
#     limg = cv2.merge((cl,a,b))
#     final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
# #     Try median filtering to remove noise
#     final = cv2.medianBlur(final,3)
    final  = light_remover(frame)
    
    # Display the resulting frame
    cv2.imshow('frame',final)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyWindow('frame')

### Remove lighting from background to get clear face image

In [59]:
def light_remover(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    lab =  cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
#     L = lab[:,:,0]
    l,a,b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(12,12))
    cl = clahe.apply(l)
#     lab = cv2.merge((cl,a,b))
    # invert the image lighting
    Medianblur = cv2.medianBlur(cl,99)
    invertedL = cv2.bitwise_not(Medianblur)
    composed = cv2.addWeighted(gray,0.75,invertedL,0.25,0)
    return composed
    

In [54]:
cap.release()